In [1]:
import pandas as pd
import numpy as np
import torch
from sklearn import preprocessing
from dgl.data import DGLDataset
import dgl
import time
import networkx as nx
import category_encoders as ce
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import torch
import tqdm
import math

from typing import *
from sklearn.preprocessing import StandardScaler, Normalizer
import socket
import struct
import random
from sklearn.model_selection import train_test_split

/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
file_name = "NF-UNSW-NB15-v2.parquet"
data = pd.read_parquet(file_name)

In [3]:
# ADDED
file_name_test = "NF-UNSW-NB15-v3.parquet"
test_data = pd.read_parquet(file_name_test)
##

In [4]:
data.Label.value_counts()

Label
0    2295222
1      95053
Name: count, dtype: int64

In [5]:
# ADDED
test_data.Label.value_counts()
##

Label
0    2151027
1      91904
Name: count, dtype: int64

In [6]:
data.rename(columns=lambda x: x.strip(), inplace=True)
data['IPV4_SRC_ADDR'] = data["IPV4_SRC_ADDR"].apply(str)
data['L4_SRC_PORT'] = data["L4_SRC_PORT"].apply(str)
data['IPV4_DST_ADDR'] = data["IPV4_DST_ADDR"].apply(str)
data['L4_DST_PORT'] = data["L4_DST_PORT"].apply(str)

In [7]:
# ADDED
test_data.rename(columns=lambda x: x.strip(), inplace=True)
test_data['IPV4_SRC_ADDR'] = test_data["IPV4_SRC_ADDR"].apply(str)
test_data['L4_SRC_PORT'] = test_data["L4_SRC_PORT"].apply(str)
test_data['IPV4_DST_ADDR'] = test_data["IPV4_DST_ADDR"].apply(str)
test_data['L4_DST_PORT'] = test_data["L4_DST_PORT"].apply(str)
##

In [8]:
data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)

In [9]:
##ADDED
test_data.drop(columns=["L4_SRC_PORT", "L4_DST_PORT"], inplace=True)
##

In [10]:
data.Attack.unique()

array(['Benign', 'Exploits', 'Generic', 'Fuzzers', 'Backdoor', 'DoS',
       'Reconnaissance', 'Shellcode', 'Worms', 'Analysis'], dtype=object)

In [11]:
##ADDED
test_data.Attack.unique()
##

array(['Benign', 'Fuzzers', 'Exploits', 'Backdoor', 'Generic', 'DoS',
       'Reconnaissance', 'Shellcode', 'Analysis', 'Worms'], dtype=object)

In [12]:
data = data.groupby(by='Attack').sample(frac=0.1, random_state=13)

In [13]:
# ADDED
test_data = test_data.groupby(by='Attack').sample(frac=0.1, random_state=13)
##

In [14]:
data.groupby(by="Attack").count()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Analysis,230,230,230,230,230,230,230,230,230,230,...,230,230,230,230,230,230,230,230,230,230
Backdoor,217,217,217,217,217,217,217,217,217,217,...,217,217,217,217,217,217,217,217,217,217
Benign,229522,229522,229522,229522,229522,229522,229522,229522,229522,229522,...,229522,229522,229522,229522,229522,229522,229522,229522,229522,229522
DoS,579,579,579,579,579,579,579,579,579,579,...,579,579,579,579,579,579,579,579,579,579
Exploits,3155,3155,3155,3155,3155,3155,3155,3155,3155,3155,...,3155,3155,3155,3155,3155,3155,3155,3155,3155,3155
Fuzzers,2231,2231,2231,2231,2231,2231,2231,2231,2231,2231,...,2231,2231,2231,2231,2231,2231,2231,2231,2231,2231
Generic,1656,1656,1656,1656,1656,1656,1656,1656,1656,1656,...,1656,1656,1656,1656,1656,1656,1656,1656,1656,1656
Reconnaissance,1278,1278,1278,1278,1278,1278,1278,1278,1278,1278,...,1278,1278,1278,1278,1278,1278,1278,1278,1278,1278
Shellcode,143,143,143,143,143,143,143,143,143,143,...,143,143,143,143,143,143,143,143,143,143


In [15]:
# ADDED
test_data.groupby(by="Attack").count()
##

,FLOW_START_MILLISECONDS,FLOW_END_MILLISECONDS,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,...,FTP_COMMAND_RET_CODE,SRC_TO_DST_IAT_MIN,SRC_TO_DST_IAT_MAX,SRC_TO_DST_IAT_AVG,SRC_TO_DST_IAT_STDDEV,DST_TO_SRC_IAT_MIN,DST_TO_SRC_IAT_MAX,DST_TO_SRC_IAT_AVG,DST_TO_SRC_IAT_STDDEV,Label
Attack,,,,,,,,,,,,,,,,,,,,,
Analysis,123,123,123,123,123,123,123,123,123,123,...,123,123,123,123,123,123,123,123,123,123
Backdoor,345,345,345,345,345,345,345,345,345,345,...,345,345,345,345,345,345,345,345,345,345
Benign,215103,215103,215103,215103,215103,215103,215103,215103,215103,215103,...,215103,215103,215103,215103,215103,215103,215103,215103,215103,215103
DoS,505,505,505,505,505,505,505,505,505,505,...,505,505,505,505,505,505,505,505,505,505
Exploits,3882,3882,3882,3882,3882,3882,3882,3882,3882,3882,...,3882,3882,3882,3882,3882,3882,3882,3882,3882,3882
Fuzzers,2559,2559,2559,2559,2559,2559,2559,2559,2559,2559,...,2559,2559,2559,2559,2559,2559,2559,2559,2559,2559
Generic,476,476,476,476,476,476,476,476,476,476,...,476,476,476,476,476,476,476,476,476,476
Reconnaissance,1129,1129,1129,1129,1129,1129,1129,1129,1129,1129,...,1129,1129,1129,1129,1129,1129,1129,1129,1129,1129
Shellcode,159,159,159,159,159,159,159,159,159,159,...,159,159,159,159,159,159,159,159,159,159


In [16]:
X = data.drop(columns=["Attack", "Label"])
y = data[["Attack", "Label"]]

# X_train, X_test, y_train, y_test = train_test_split(
#         X, y, test_size=0.3, random_state=13, stratify=y)
# ADDED
X_train, y_train = X, y
X_test = test_data.drop(columns=["Attack", "Label", "FLOW_START_MILLISECONDS", "FLOW_END_MILLISECONDS",
                       "SRC_TO_DST_IAT_MIN", "SRC_TO_DST_IAT_MAX", "SRC_TO_DST_IAT_AVG",
                       "SRC_TO_DST_IAT_STDDEV", "DST_TO_SRC_IAT_MIN", "DST_TO_SRC_IAT_MAX",
                       "DST_TO_SRC_IAT_AVG", "DST_TO_SRC_IAT_STDDEV"])
y_test = test_data[["Attack", "Label"]]

X_train.shape, X_test.shape, y_train.shape, y_test.shape
##

((239027, 41), (224295, 41), (239027, 2), (224295, 2))

In [17]:
encoder = ce.TargetEncoder(cols=['TCP_FLAGS','L7_PROTO','PROTOCOL',
                                  'CLIENT_TCP_FLAGS','SERVER_TCP_FLAGS','ICMP_TYPE',
                                  'ICMP_IPV4_TYPE','DNS_QUERY_ID','DNS_QUERY_TYPE',
                                  'FTP_COMMAND_RET_CODE'])
encoder.fit(X_train, y_train.Label)

# Transform on training set
X_train = encoder.transform(X_train)

# Transform on testing set
X_test = encoder.transform(X_test)

In [18]:
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)
X_test.replace([np.inf, -np.inf], np.nan, inplace=True)
X_train.fillna(0, inplace=True)
X_test.fillna(0, inplace=True)

In [19]:
scaler = Normalizer()
cols_to_norm = list(set(list(X_train.iloc[:, 2:].columns))) # Ignore first two as the represents IP addresses
scaler.fit(X_train[cols_to_norm])

# Transform on training set
X_train[cols_to_norm] = scaler.transform(X_train[cols_to_norm])
X_train['h'] = X_train.iloc[:, 2:].values.tolist()

# Transform on testing set
X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
X_test['h'] = X_test.iloc[:, 2:].values.tolist()

train = pd.concat([X_train, y_train], axis=1)
test = pd.concat([X_test, y_test], axis=1)

In [20]:
X_train.head()

,IPV4_SRC_ADDR,IPV4_DST_ADDR,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,...,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,h
1290726,175.45.176.2,149.171.126.17,1.343354e-07,2.443527e-08,0.000125,1.250000e-06,0.000000,0.000000e+00,4.211642e-08,4.211339e-08,...,0.0,0.000000,0.000000,3.555351e-08,3.557049e-08,2.577007e-08,2.577316e-08,0.0,2.796590e-08,"[1.343354220351684e-07, 2.443527253424305e-08,..."
1558308,175.45.176.0,149.171.126.17,1.637724e-07,2.443527e-08,0.000125,1.250000e-06,0.000000,0.000000e+00,4.211642e-08,4.211339e-08,...,0.0,0.000000,0.000000,3.555351e-08,3.557049e-08,2.577007e-08,2.577316e-08,0.0,2.796590e-08,"[1.6377242873589795e-07, 2.4435272534242944e-0..."
1134643,175.45.176.0,149.171.126.14,2.769462e-09,7.564262e-09,0.000071,8.824831e-07,0.000103,7.059865e-07,1.092340e-09,1.169842e-09,...,0.0,0.001446,0.001446,1.709522e-08,1.737531e-08,3.638665e-09,3.639101e-09,0.0,3.948710e-09,"[2.7694615615475404e-09, 7.564261618940174e-09..."
1294741,175.45.176.2,149.171.126.17,1.533838e-07,2.443527e-08,0.000125,1.250000e-06,0.000000,0.000000e+00,4.211642e-08,4.211339e-08,...,0.0,0.000000,0.000000,3.555351e-08,3.557049e-08,2.577007e-08,2.577316e-08,0.0,2.796590e-08,"[1.533837711937069e-07, 2.4435272534242984e-08..."
1134430,175.45.176.0,149.171.126.14,7.301136e-09,1.994168e-08,0.000392,5.118282e-06,0.000520,4.187685e-06,2.879736e-09,3.084057e-09,...,0.0,0.003811,0.003811,4.191877e-09,5.539266e-09,9.592618e-09,9.593768e-09,0.0,1.040999e-08,"[7.301136175771591e-09, 1.994167563682817e-08,..."


In [21]:
lab_enc = preprocessing.LabelEncoder()
lab_enc.fit(data["Attack"])

# Transform on training set
train["Attack"] = lab_enc.transform(train["Attack"])

# Transform on testing set
test["Attack"] = lab_enc.transform(test["Attack"])

In [22]:
# Training graph

train_g = nx.from_pandas_edgelist(train, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
           ["h", "Label", "Attack"], create_using=nx.MultiGraph())
print(train_g) 
train_g = train_g.to_directed()
print(train_g) 
train_g = dgl.from_networkx(train_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([train_g.number_of_nodes(),
train_g.edata['h'].shape[1]])
train_g.ndata['h'] = nfeat_weight

# Testing graph
test_g = nx.from_pandas_edgelist(test, "IPV4_SRC_ADDR", "IPV4_DST_ADDR",
            ["h", "Label", "Attack"], create_using=nx.MultiGraph())

test_g = test_g.to_directed()
test_g = dgl.from_networkx(test_g, edge_attrs=['h', 'Attack', 'Label'])
nfeat_weight = torch.ones([test_g.number_of_nodes(),
test_g.edata['h'].shape[1]])
test_g.ndata['h'] = nfeat_weight

MultiGraph with 40 nodes and 239027 edges
MultiDiGraph with 40 nodes and 478054 edges


In [23]:
import torch.nn as nn
import torch.nn.functional as F
import dgl.function as fn
import tqdm
import gc

class SAGELayer(nn.Module):
    def __init__(self, ndim_in, edims, ndim_out, activation):
      super(SAGELayer, self).__init__()
      self.W_apply = nn.Linear(ndim_in + edims , ndim_out)
      self.activation = F.relu
      self.W_edge = nn.Linear(128 * 2, 256)
      self.reset_parameters()

    def reset_parameters(self):
      gain = nn.init.calculate_gain('relu')
      nn.init.xavier_uniform_(self.W_apply.weight, gain=gain)

    def message_func(self, edges):
      return {'m':  edges.data['h']}

    def forward(self, g_dgl, nfeats, efeats):
      with g_dgl.local_scope():
        g = g_dgl
        g.ndata['h'] = nfeats
        g.edata['h'] = efeats
        g.update_all(self.message_func, fn.mean('m', 'h_neigh'))
        g.ndata['h'] = F.relu(self.W_apply(torch.cat([g.ndata['h'], g.ndata['h_neigh']], 2)))

        # Compute edge embeddings
        u, v = g.edges()
        edge = self.W_edge(torch.cat((g.srcdata['h'][u], g.dstdata['h'][v]), 2))
        return g.ndata['h'], edge

In [24]:
class SAGE(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim,  activation):
      super(SAGE, self).__init__()
      self.layers = nn.ModuleList()
      self.layers.append(SAGELayer(ndim_in, edim, 128, F.relu))

    def forward(self, g, nfeats, efeats, corrupt=False):
      if corrupt:
        e_perm = torch.randperm(g.number_of_edges())
        #n_perm = torch.randperm(g.number_of_nodes())
        efeats = efeats[e_perm]
        #nfeats = nfeats[n_perm]
      for i, layer in enumerate(self.layers):
        #nfeats = layer(g, nfeats, efeats)
        nfeats, e_feats = layer(g, nfeats, efeats)
      #return nfeats.sum(1)
      return nfeats.sum(1), e_feats.sum(1)

In [25]:
class Discriminator(nn.Module):
    def __init__(self, n_hidden):
      super(Discriminator, self).__init__()
      self.weight = nn.Parameter(torch.Tensor(n_hidden, n_hidden))
      self.reset_parameters()

    def uniform(self, size, tensor):
      bound = 1.0 / math.sqrt(size)
      if tensor is not None:
        tensor.data.uniform_(-bound, bound)

    def reset_parameters(self):
      size = self.weight.size(0)
      self.uniform(size, self.weight)

    def forward(self, features, summary):
      features = torch.matmul(features, torch.matmul(self.weight, summary))
      return features

In [26]:
class DGI(nn.Module):
    def __init__(self, ndim_in, ndim_out, edim, activation):
      super(DGI, self).__init__()
      self.encoder = SAGE(ndim_in, ndim_out, edim,  F.relu)
      #self.discriminator = Discriminator(128)
      self.discriminator = Discriminator(256)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)
      self.loss = nn.BCEWithLogitsLoss()

    def forward(self, g, n_features, e_features):
      positive = self.encoder(g, n_features, e_features, corrupt=False)
      negative = self.encoder(g, n_features, e_features, corrupt=True)

      positive = positive[1]
      negative = negative[1]

      summary = torch.sigmoid(positive.mean(dim=0))

      positive = self.discriminator(positive, summary)
      negative = self.discriminator(negative, summary)

      l1 = self.loss(positive, torch.ones_like(positive))
      l2 = self.loss(negative, torch.zeros_like(negative))

      return l1 + l2

In [27]:
ndim_in = train_g.ndata['h'].shape[1]
hidden_features = 128
ndim_out = 128
num_layers = 1
edim = train_g.edata['h'].shape[1]
learning_rate = 1e-3
epochs = 4000

In [28]:
dgi = DGI(ndim_in,
    ndim_out,
    edim,
    F.relu)

dgi = dgi.to('cuda')

dgi_optimizer = torch.optim.Adam(dgi.parameters(),
                lr=1e-3,
                weight_decay=0.)

In [29]:
# Format node and edge features for E-GraphSAGE
train_g.ndata['h'] = torch.reshape(train_g.ndata['h'],
                                   (train_g.ndata['h'].shape[0], 1,
                                    train_g.ndata['h'].shape[1]))

train_g.edata['h'] = torch.reshape(train_g.edata['h'],
                                   (train_g.edata['h'].shape[0], 1,
                                    train_g.edata['h'].shape[1]))

In [30]:
# Convert to GPU
train_g = train_g.to('cuda')

In [31]:
# cnt_wait = 0
# best = 1e9
# best_t = 0
# dur = []
# node_features = train_g.ndata['h'] 
# edge_features = train_g.edata['h']

# for epoch in range(epochs):
#     dgi.train()
#     if epoch >= 3:
#         t0 = time.time()

#     dgi_optimizer.zero_grad()
#     loss = dgi(train_g, node_features, edge_features)
#     loss.backward()
#     dgi_optimizer.step()

#     if loss < best:
#         best = loss
#         best_t = epoch
#         cnt_wait = 0
#         torch.save(dgi.state_dict(), 'best_dgi_UNSW.pkl')
#     else:
#         cnt_wait += 1

#   # if cnt_wait == patience:
#   #     print('Early stopping!')
#   #     break

#     if epoch >= 3:
#         dur.append(time.time() - t0)

#     if epoch % 50 == 0:

#         print("Epoch {:05d} | Time(s) {:.4f} | Loss {:.4f} | "
#             "ETputs(KTEPS) {:.2f}".format(epoch, np.mean(dur),
#               loss.item(),
#               train_g.num_edges() / np.mean(dur) / 1000))

In [32]:
dgi.load_state_dict(torch.load('best_dgi_UNSW.pkl'))

<All keys matched successfully>

In [33]:
training_emb = dgi.encoder(train_g, train_g.ndata['h'], train_g.edata['h'])[1]
training_emb = training_emb.detach().cpu().numpy()

In [34]:
test_g.ndata['h'] = torch.reshape(test_g.ndata['h'],
                                   (test_g.ndata['h'].shape[0], 1,
                                    test_g.ndata['h'].shape[1]))



test_g.edata['h'] = torch.reshape(test_g.edata['h'],
                                   (test_g.edata['h'].shape[0], 1,
                                    test_g.edata['h'].shape[1]))

In [35]:
# Convert to GPU
test_g = test_g.to('cuda')

In [36]:
testing_emb = dgi.encoder(test_g, test_g.ndata['h'], test_g.edata['h'])[1]
testing_emb = testing_emb.detach().cpu().numpy()

In [37]:
df_train = pd.DataFrame(training_emb, )
df_train["Attack"] = lab_enc.inverse_transform(
        train_g.edata['Attack'].detach().cpu().numpy())
df_train["Label"] = train_g.edata['Label'].detach().cpu().numpy()

df_test = pd.DataFrame(testing_emb, )
df_test["Attack"] = lab_enc.inverse_transform(
        test_g.edata['Attack'].detach().cpu().numpy())
df_test["Label"] = test_g.edata['Label'].detach().cpu().numpy()

In [38]:
df_train

,0,1,2,3,4,5,6,7,8,9,...,248,249,250,251,252,253,254,255,Attack,Label
0,-0.042512,0.049039,-0.005300,0.065045,-0.012278,0.040998,-0.042536,0.011380,-0.006963,0.043847,...,0.001644,-0.012790,-0.027071,0.000859,0.060748,0.049902,-0.041244,0.031112,Analysis,1
1,-0.042512,0.049039,-0.005300,0.065045,-0.012278,0.040998,-0.042536,0.011380,-0.006963,0.043847,...,0.001644,-0.012790,-0.027071,0.000859,0.060748,0.049902,-0.041244,0.031112,Analysis,1
2,-0.042512,0.049039,-0.005300,0.065045,-0.012278,0.040998,-0.042536,0.011380,-0.006963,0.043847,...,0.001644,-0.012790,-0.027071,0.000859,0.060748,0.049902,-0.041244,0.031112,Analysis,1
3,-0.042512,0.049039,-0.005300,0.065045,-0.012278,0.040998,-0.042536,0.011380,-0.006963,0.043847,...,0.001644,-0.012790,-0.027071,0.000859,0.060748,0.049902,-0.041244,0.031112,Analysis,1
4,-0.042512,0.049039,-0.005300,0.065045,-0.012278,0.040998,-0.042536,0.011380,-0.006963,0.043847,...,0.001644,-0.012790,-0.027071,0.000859,0.060748,0.049902,-0.041244,0.031112,Analysis,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478049,0.025993,0.183050,-0.072745,0.100478,-0.055202,0.094599,-0.092061,0.023950,0.029100,0.138175,...,0.029412,0.036141,-0.050864,0.159174,0.104053,0.082916,0.018884,0.062048,Benign,0
478050,0.025993,0.183050,-0.072745,0.100478,-0.055202,0.094599,-0.092061,0.023950,0.029100,0.138175,...,0.029412,0.036141,-0.050864,0.159174,0.104053,0.082916,0.018884,0.062048,Benign,0
478051,0.025993,0.183050,-0.072745,0.100478,-0.055202,0.094599,-0.092061,0.023950,0.029100,0.138175,...,0.029412,0.036141,-0.050864,0.159174,0.104053,0.082916,0.018884,0.062048,Benign,0
478052,-0.002630,0.153330,-0.043562,0.087384,-0.017167,0.065387,-0.031002,0.007393,-0.015916,0.119692,...,-0.003777,-0.007802,-0.008208,0.086704,0.069430,0.083937,-0.015014,0.052698,Benign,0


# Embeddings CBLOF  Embeddings

In [39]:
import torch
import dgl
import numpy as np
import pandas as pd
import torch.optim as optim
import time
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, f1_score
from sklearn.ensemble import IsolationForest
import gc

from tqdm import tqdm
import itertools

In [40]:
benign_train_samples = df_train[df_train.Label == 0].drop(columns=["Label", "Attack"])
normal_train_samples = df_train.drop(columns=["Label", "Attack"])

train_labels = df_train["Label"]
test_labels = df_test["Label"]

test_samples = df_test.drop(columns=["Label", "Attack"])

In [41]:
test_samples

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,-0.013382,0.063569,-0.033521,0.063758,-0.033742,0.045809,-0.065304,0.005479,-0.009770,0.064721,...,-0.013684,0.080198,0.008691,-0.006861,-0.024916,0.030827,0.071382,0.064631,-0.044821,0.050472
1,-0.013382,0.063569,-0.033521,0.063758,-0.033742,0.045809,-0.065304,0.005479,-0.009770,0.064721,...,-0.013684,0.080198,0.008691,-0.006861,-0.024916,0.030827,0.071382,0.064631,-0.044821,0.050472
2,-0.013382,0.063569,-0.033521,0.063758,-0.033742,0.045809,-0.065304,0.005479,-0.009770,0.064721,...,-0.013684,0.080198,0.008691,-0.006861,-0.024916,0.030827,0.071382,0.064631,-0.044821,0.050472
3,-0.013382,0.063569,-0.033521,0.063758,-0.033742,0.045809,-0.065304,0.005479,-0.009770,0.064721,...,-0.013684,0.080198,0.008691,-0.006861,-0.024916,0.030827,0.071382,0.064631,-0.044821,0.050472
4,-0.013382,0.063569,-0.033521,0.063758,-0.033742,0.045809,-0.065304,0.005479,-0.009770,0.064721,...,-0.013684,0.080198,0.008691,-0.006861,-0.024916,0.030827,0.071382,0.064631,-0.044821,0.050472
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448580,0.024852,0.112039,-0.113054,0.131248,-0.062647,0.056990,-0.058300,0.017559,0.027202,0.105126,...,-0.007418,0.096166,-0.005434,0.025682,-0.050716,0.131188,0.076499,0.058874,-0.003630,0.067461
448581,0.024852,0.112039,-0.113054,0.131248,-0.062647,0.056990,-0.058300,0.017559,0.027202,0.105126,...,-0.007418,0.096166,-0.005434,0.025682,-0.050716,0.131188,0.076499,0.058874,-0.003630,0.067461
448582,0.024852,0.112039,-0.113054,0.131248,-0.062647,0.056990,-0.058300,0.017559,0.027202,0.105126,...,-0.007418,0.096166,-0.005434,0.025682,-0.050716,0.131188,0.076499,0.058874,-0.003630,0.067461
448583,0.024852,0.112039,-0.113054,0.131248,-0.062647,0.056990,-0.058300,0.017559,0.027202,0.105126,...,-0.007418,0.096166,-0.005434,0.025682,-0.050716,0.131188,0.076499,0.058874,-0.003630,0.067461


In [42]:
df_raw_train = pd.concat([X_train.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_train], axis=1)
df_raw_test = pd.concat([X_test.drop(columns=["IPV4_SRC_ADDR","IPV4_DST_ADDR", "h"]), y_test], axis=1)

In [43]:
df_raw_train

,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Attack,Label
1290726,1.343354e-07,2.443527e-08,0.000125,1.250000e-06,0.000000,0.000000e+00,4.211642e-08,4.211339e-08,4.365125e-08,0.000000,...,0.000000,0.000000,3.555351e-08,3.557049e-08,2.577007e-08,2.577316e-08,0.0,2.796590e-08,Analysis,1
1558308,1.637724e-07,2.443527e-08,0.000125,1.250000e-06,0.000000,0.000000e+00,4.211642e-08,4.211339e-08,4.365125e-08,0.000000,...,0.000000,0.000000,3.555351e-08,3.557049e-08,2.577007e-08,2.577316e-08,0.0,2.796590e-08,Analysis,1
1134643,2.769462e-09,7.564262e-09,0.000071,8.824831e-07,0.000103,7.059865e-07,1.092340e-09,1.169842e-09,4.214778e-08,0.000000,...,0.001446,0.001446,1.709522e-08,1.737531e-08,3.638665e-09,3.639101e-09,0.0,3.948710e-09,Analysis,1
1294741,1.533838e-07,2.443527e-08,0.000125,1.250000e-06,0.000000,0.000000e+00,4.211642e-08,4.211339e-08,4.365125e-08,0.000000,...,0.000000,0.000000,3.555351e-08,3.557049e-08,2.577007e-08,2.577316e-08,0.0,2.796590e-08,Analysis,1
1134430,7.301136e-09,1.994168e-08,0.000392,5.118282e-06,0.000520,4.187685e-06,2.879736e-09,3.084057e-09,1.111143e-07,0.999191,...,0.003811,0.003811,4.191877e-09,5.539266e-09,9.592618e-09,9.593768e-09,0.0,1.040999e-08,Analysis,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2257034,2.929995e-09,8.002729e-09,0.000121,9.336368e-07,0.000031,7.469094e-07,7.910815e-08,2.397258e-08,4.459091e-08,0.000000,...,0.001530,0.001530,9.172572e-09,9.759269e-09,3.849582e-09,3.850044e-09,0.0,4.177599e-09,Worms,1
1542232,4.522625e-10,1.235270e-09,0.000125,2.017575e-07,0.000004,8.646750e-08,1.221082e-08,3.700313e-09,6.882877e-09,0.000000,...,0.000236,0.000236,4.730347e-09,4.803707e-09,5.942063e-10,5.942776e-10,0.0,6.448377e-10,Worms,1
1832562,2.969486e-09,8.110590e-09,0.000123,9.462205e-07,0.000024,5.677323e-07,8.017438e-08,2.429568e-08,4.519191e-08,0.000000,...,0.001550,0.001550,9.296201e-09,9.890806e-09,3.901467e-09,3.901935e-09,0.0,4.233905e-09,Worms,1
2265613,2.955631e-09,8.072748e-09,0.000121,9.418055e-07,0.000032,7.534444e-07,7.980030e-08,2.418232e-08,4.498105e-08,0.000000,...,0.001543,0.001543,6.058805e-08,6.093500e-08,3.883264e-09,3.883729e-09,0.0,4.214150e-09,Worms,1


In [44]:
df_raw_test

,PROTOCOL,L7_PROTO,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,TCP_FLAGS,CLIENT_TCP_FLAGS,SERVER_TCP_FLAGS,FLOW_DURATION_MILLISECONDS,...,TCP_WIN_MAX_IN,TCP_WIN_MAX_OUT,ICMP_TYPE,ICMP_IPV4_TYPE,DNS_QUERY_ID,DNS_QUERY_TYPE,DNS_TTL_ANSWER,FTP_COMMAND_RET_CODE,Attack,Label
28228,1.069293e-06,2.920571e-06,0.028485,0.000341,0.045794,0.000341,4.217537e-07,4.516775e-07,1.627331e-05,0.024089,...,0.558214,0.558214,3.331732e-06,3.662182e-06,1.404893e-06,1.405062e-06,0.0,1.524602e-06,Analysis,1
856405,8.580243e-07,2.343531e-06,0.022255,0.000273,0.035816,0.000219,3.384246e-07,3.624361e-07,1.305807e-05,0.011921,...,0.447924,0.447924,1.252246e-06,1.252246e-06,1.127318e-06,1.127453e-06,0.0,1.223375e-06,Analysis,1
465045,1.235825e-06,3.375423e-06,0.031897,0.000394,0.043554,0.000315,4.874379e-07,5.220221e-07,1.880773e-05,0.042687,...,0.645151,0.645151,6.893922e-06,7.650934e-06,1.623692e-06,1.623887e-06,0.0,1.762045e-06,Analysis,1
959848,1.159611e-06,3.167257e-06,0.029782,0.000370,0.047666,0.000370,4.573771e-07,4.898285e-07,1.764784e-05,0.032554,...,0.605364,0.605364,1.640142e-05,1.636078e-05,1.523558e-06,1.523740e-06,0.0,1.653377e-06,Analysis,1
1381572,1.100709e-06,3.006379e-06,0.028129,0.000351,0.038160,0.000281,4.341450e-07,4.649480e-07,1.675143e-05,0.022938,...,0.574615,0.574615,1.077942e-05,1.234439e-05,1.446170e-06,1.446343e-06,0.0,1.569395e-06,Analysis,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
442625,7.723334e-07,2.109483e-06,0.031895,0.000246,0.006300,0.000148,2.085255e-05,6.319063e-06,1.175396e-05,0.007777,...,0.403190,0.403190,8.477061e-06,8.381610e-06,1.014732e-06,1.014854e-06,0.0,1.101196e-06,Worms,1
1417188,1.146568e-06,3.131633e-06,0.047350,0.000365,0.009353,0.000219,3.095665e-05,9.380965e-06,1.744934e-05,0.026707,...,0.598555,0.598555,1.258462e-05,1.244292e-05,1.506421e-06,1.506602e-06,0.0,1.634781e-06,Worms,1
892190,4.850304e-07,1.324769e-06,0.020030,0.000155,0.003957,0.000093,1.309554e-05,3.968412e-06,7.381563e-06,0.002689,...,0.253206,0.253206,5.323649e-06,5.263705e-06,6.372585e-07,6.373349e-07,0.0,6.915583e-07,Worms,1
763320,1.254539e-06,3.426536e-06,0.051728,0.000400,0.013432,0.000320,3.387181e-05,1.026436e-05,1.909253e-05,0.047051,...,0.654920,0.654920,6.012453e-06,6.518332e-06,1.648279e-06,1.648477e-06,0.0,1.788726e-06,Worms,1


In [45]:
raw_benign_train_samples = df_raw_train[df_raw_train.Label == 0].drop(columns=["Label", "Attack"])
raw_normal_train_samples = df_raw_train.drop(columns=["Label", "Attack"])

raw_train_labels = df_raw_train["Label"]
raw_test_labels = df_raw_test["Label"]

raw_test_samples = df_raw_test.drop(columns=["Label", "Attack"])

In [46]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [47]:
from pyod.models.cblof import CBLOF
n_est = [2,3,5,7,9,10]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [02:13<00:00,  3.72s/it]

{'n_estimators': 2, 'con': 0.001}
0.9950400580872917
              precision    recall  f1-score   support

           0     1.0000    0.9992    0.9996    430201
           1     0.9812    1.0000    0.9905     18384

    accuracy                         0.9992    448585
   macro avg     0.9906    0.9996    0.9950    448585
weighted avg     0.9992    0.9992    0.9992    448585



In [48]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = CBLOF(n_clusters=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:43<00:00,  2.87s/it]

{'n_estimators': 2, 'con': 0.001}
0.9950400580872917
              precision    recall  f1-score   support

           0     1.0000    0.9992    0.9996    430201
           1     0.9812    1.0000    0.9905     18384

    accuracy                         0.9992    448585
   macro avg     0.9906    0.9996    0.9950    448585
weighted avg     0.9992    0.9992    0.9992    448585



In [49]:
###  CBLOF RAW

In [50]:
from pyod.models.cblof import CBLOF

n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = CBLOF(n_clusters=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  3%|▎         | 1/36 [00:00<00:07,  4.45it/s]

2


  6%|▌         | 2/36 [00:00<00:07,  4.62it/s]

2


 11%|█         | 4/36 [00:00<00:06,  4.83it/s]

...

3


 31%|███       | 11/36 [00:03<00:08,  3.04it/s]

3


100%|██████████| 36/36 [00:19<00:00,  1.81it/s]

{'n_estimators': 3, 'con': 0.01}
0.7175246137607729
              precision    recall  f1-score   support

           0     0.9754    0.9819    0.9787    215103
           1     0.4987    0.4207    0.4564      9192

    accuracy                         0.9589    224295
   macro avg     0.7371    0.7013    0.7175    224295
weighted avg     0.9559    0.9589    0.9573    224295



In [51]:
n_est = [2,3,5,7,9,10]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = CBLOF(n_clusters=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

  6%|▌         | 2/36 [00:00<00:06,  5.12it/s]

2
2


  8%|▊         | 3/36 [00:00<00:06,  5.06it/s]

2


 17%|█▋        | 6/36 [00:01<00:07,  3.85it/s]

2
2


 31%|███       | 11/36 [00:04<00:10,  2.42it/s]

3


100%|██████████| 36/36 [00:24<00:00,  1.48it/s]

benign only
{'n_estimators': 2}
0.7261147625263039
              precision    recall  f1-score   support

           0     0.9903    0.9388    0.9639    215103
           1     0.3542    0.7859    0.4884      9192

    accuracy                         0.9325    224295
   macro avg     0.6723    0.8623    0.7261    224295
weighted avg     0.9643    0.9325    0.9444    224295



In [52]:
# HBOS  Embeddings

In [53]:
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]

In [54]:
from pyod.models.hbos import HBOS

n_est = [5,10,15,20,25,30]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [03:56<00:00,  6.56s/it]

{'n_estimators': 5, 'con': 0.001}
0.03936878036871826
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    430201
           1     0.0410    1.0000    0.0787     18384

    accuracy                         0.0410    448585
   macro avg     0.0205    0.5000    0.0394    448585
weighted avg     0.0017    0.0410    0.0032    448585




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [55]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    clf_if = HBOS(n_bins=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [04:03<00:00,  6.77s/it]

{'n_estimators': 5, 'con': 0.001}
0.03936878036871826
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    430201
           1     0.0410    1.0000    0.0787     18384

    accuracy                         0.0410    448585
   macro avg     0.0205    0.5000    0.0394    448585
weighted avg     0.0017    0.0410    0.0032    448585




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [56]:
##  HBOS  RAw

In [57]:
from pyod.models.cblof import CBLOF

n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_b = HBOS(n_bins=n_est, contamination=con)
        clf_b.fit(raw_benign_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
   
    y_pred = clf_b.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_b
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:20<00:00,  1.72it/s]

{'n_estimators': 5, 'con': 0.001}
0.4376112310497001
              precision    recall  f1-score   support

           0     1.0000    0.5553    0.7141    215103
           1     0.0876    0.9995    0.1611      9192

    accuracy                         0.5735    224295
   macro avg     0.5438    0.7774    0.4376    224295
weighted avg     0.9626    0.5735    0.6914    224295



In [58]:
n_est = [5,10,15,20,25,30]
contamination = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, contamination))
score = -1
bs = None
for n_est, con in tqdm(params):
    
    try:
        clf_if = HBOS(n_bins=n_est, contamination=con)
        clf_if.fit(raw_normal_train_samples)
    except ValueError as e:
        print(n_est)
        continue  
    
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:20<00:00,  1.74it/s]

benign only
{'n_estimators': 10}
0.4437017926680764
              precision    recall  f1-score   support

           0     1.0000    0.5661    0.7229    215103
           1     0.0896    0.9995    0.1645      9192

    accuracy                         0.5839    224295
   macro avg     0.5448    0.7828    0.4437    224295
weighted avg     0.9627    0.5839    0.7000    224295



In [59]:
##  PCA  Emb

In [60]:
from pyod.models.pca import PCA
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:35<00:00,  2.66s/it]

{'n_estimators': 5, 'con': 0.001}
0.03936878036871826
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    430201
           1     0.0410    1.0000    0.0787     18384

    accuracy                         0.0410    448585
   macro avg     0.0205    0.5000    0.0394    448585
weighted avg     0.0017    0.0410    0.0032    448585




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [61]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = y_pred

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 36/36 [01:39<00:00,  2.77s/it]

{'n_estimators': 5, 'con': 0.001}
0.03936878036871826
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000    430201
           1     0.0410    1.0000    0.0787     18384

    accuracy                         0.0410    448585
   macro avg     0.0205    0.5000    0.0394    448585
weighted avg     0.0017    0.0410    0.0032    448585




/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/kienho/projects/NIDS_Anomal-E_revision/.conda/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
 

In [62]:
##  PCA  RAw

In [63]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples)
   
    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:15<00:00,  2.38it/s]

{'n_estimators': 5, 'con': 0.001}
0.4353518341059648
              precision    recall  f1-score   support

           0     1.0000    0.5513    0.7107    215103
           1     0.0869    1.0000    0.1600      9192

    accuracy                         0.5696    224295
   macro avg     0.5435    0.7756    0.4354    224295
weighted avg     0.9626    0.5696    0.6881    224295



In [64]:
n_est = [5,10,15,20,25,30]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = PCA(n_components=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples)

    y_pred = clf_if.predict(raw_test_samples)
    test_pred = y_pred

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 36/36 [00:15<00:00,  2.29it/s]

benign only
{'n_estimators': 5}
0.45016496748158014
              precision    recall  f1-score   support

           0     1.0000    0.5774    0.7321    215103
           1     0.0918    0.9999    0.1682      9192

    accuracy                         0.5947    224295
   macro avg     0.5459    0.7887    0.4502    224295
weighted avg     0.9628    0.5947    0.7090    224295



In [65]:
##  IF  Emb

In [66]:
from sklearn.ensemble import IsolationForest
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(benign_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [01:05<00:00,  2.75s/it]

{'n_estimators': 20, 'con': 0.01}
0.7011514606777126
              precision    recall  f1-score   support

           0     0.9695    1.0000    0.9845    430201
           1     0.9965    0.2643    0.4178     18384

    accuracy                         0.9698    448585
   macro avg     0.9830    0.6321    0.7012    448585
weighted avg     0.9706    0.9698    0.9613    448585



In [67]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(normal_train_samples)
    y_pred = clf_if.predict(test_samples)
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                       "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()


print(best_params)
print(score)
print(classification_report(test_labels, bs, digits=4))

100%|██████████| 24/24 [01:05<00:00,  2.74s/it]

{'n_estimators': 50, 'con': 0.01}
0.9991505325571275
              precision    recall  f1-score   support

           0     1.0000    0.9999    0.9999    430201
           1     0.9967    1.0000    0.9984     18384

    accuracy                         0.9999    448585
   macro avg     0.9984    0.9999    0.9992    448585
weighted avg     0.9999    0.9999    0.9999    448585



In [68]:
##  IF  Raw

In [69]:
from sklearn.ensemble import IsolationForest

n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_benign_train_samples.to_numpy())
   
    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est,
                        "con": con
                }
        bs = test_pred
    del clf_if
    gc.collect()

  

print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:23<00:00,  1.03it/s]

{'n_estimators': 100, 'con': 0.001}
0.44225435823269965
              precision    recall  f1-score   support

           0     0.9963    0.5710    0.7260    215103
           1     0.0865    0.9505    0.1586      9192

    accuracy                         0.5866    224295
   macro avg     0.5414    0.7608    0.4423    224295
weighted avg     0.9590    0.5866    0.7027    224295



In [70]:
n_est = [20, 50, 100, 150]
cont = [0.001, 0.01, 0.04, 0.05, 0.1, 0.2]
params = list(itertools.product(n_est, cont))
score = -1
bs = None

for n_est, con in tqdm(params):
    clf_if = IsolationForest(n_estimators=n_est, contamination=con)
    clf_if.fit(raw_normal_train_samples.to_numpy())

    y_pred = clf_if.predict(raw_test_samples.to_numpy())
    test_pred = list(map(lambda x : 0 if x == 1 else 1, y_pred))

    f1 = f1_score(raw_test_labels, test_pred, average='macro')

    if f1 > score:
        score = f1
        best_params = {'n_estimators': n_est
                }
        bs = test_pred
    del clf_if
    gc.collect()

        

print("benign only")
print(best_params)
print(score)
print(classification_report(raw_test_labels, bs, digits=4))

100%|██████████| 24/24 [00:23<00:00,  1.02it/s]

benign only
{'n_estimators': 20}
0.4647756836435692
              precision    recall  f1-score   support

           0     0.9965    0.6112    0.7577    215103
           1     0.0945    0.9495    0.1719      9192

    accuracy                         0.6251    224295
   macro avg     0.5455    0.7804    0.4648    224295
weighted avg     0.9595    0.6251    0.7337    224295

